### 1. Vector database

Get information from vector database

In [2]:
from tqdm.auto import tqdm
from RAG_BASIC.utils.utils import load_structure
from RAG_BASIC.chromadbClass.chromadb import Chromadb_Class
import os

In [3]:
current_path = os.getcwd()
db_path = os.path.join(current_path, 'vector_db')
model = "llama3.1"
collection_name = "doc_test"

In [4]:
db = Chromadb_Class(path=db_path, model=model, collection_name=collection_name)

Define Query

In [5]:
query = "¿Cuál es el objetivo principal del Código Orgánico de la Producción, Comercio e Inversiones? "
n_results = 10
retrieval_vector_db = db.search_data_collection(query, n_results=n_results)

In [6]:
retrieval_vector_db.keys()

dict_keys(['ids', 'distances', 'metadatas', 'embeddings', 'documents', 'uris', 'data'])

In [7]:
retrieval_vector_db_documents = retrieval_vector_db['documents'][0]

In [8]:
retrieval_vector_db_documents

['a.\nLas responsabilidades de las personas autorizadas para el funcionamiento de las bodegas destinadas a depósito temporal y depósitos aduaneros, son las siguientes:\n1.\nIndemnizar al dueño o consignatario de la carga por los daños soportados por la destrucción o pérdida de su mercancía.\n2.\nPagar al Estado los tributos correspondientes.\nEsta responsabilidad se extiende a los tributos que hubieren correspondido a las mercancías que sufran cualquier siniestro, robo o hurto durante su traslado desde el puerto, aeropuerto o frontera de arribo, hasta las bodegas de depósito.\nb.\nEs responsabilidad del dueño, consignatario o el consignante de las mercancías indemnizar por los daños y perjuicios causados en las bodegas, por la naturaleza o peligro de sus mercancías, cuando no hubiere manifestado estas condiciones en los documentos de embarque, o informado expresamente de ellas a los propietarios o empresas autorizadas para bodegas destinadas a depósito temporal y depósitos aduaneros.\n

### 2. BM25 sparse retrieval

In [9]:
from RAG_BASIC.BM25_rank.BM25_class import BM25_score

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [10]:
k_items = 5
bm25 = BM25_score(load=True, path_load='./BM25_retriever')
retrieval_bm25 =bm25.__search__(query=query, k=k_items)

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
retrieval_bm25_documents  = [item['text'] for item in retrieval_bm25[0][0]]

In [16]:
retrieval_bm25_documents

['Nota:\nLas facultades determinadas en el artículo 102 del Código Orgánico de la Producción, Comercio e Inversiones, que correspondían al Ministerio de Industrias y Productividad, a partir de la fecha de expedición del presente Decreto Ejecutivo, serán ejercidas por el Instituto Nacional de Contratación Pública.\nDado por Decreto Ejecutivo No. 1516, publicado en Registro Oficial Suplemento 5 de 31 de Mayo del 2013 .\nCAPITULO !l\nLEXISFIND\nDEL FOMENTO Y PROMOCION DE LAS INDUSTRIAS BASICAS\nArt. ...- Sin perjuicio de los incentivos para el desarrollo productivo y fomento a las exportaciones establecidas en el presente Código, el Estado promocionará las industrias básicas que aprovechen las materias primas provenientes de los recursos naturales renovables y no renovables, transformándolos en productos que luego sean requeridos por otras industrias para la fabricación de productos intermedios y finales.',
 'LEXISFIND\nPagos Interbancarios (SPI) en el Banco Central del Ecuador, desde la 

### 3. Combine candidates from dense (vector database) and sparse (BM25)

In [17]:
retrieval_vector_db_documents.extend(retrieval_bm25_documents)

In [18]:
retrieval_vector_db_documents

['a.\nLas responsabilidades de las personas autorizadas para el funcionamiento de las bodegas destinadas a depósito temporal y depósitos aduaneros, son las siguientes:\n1.\nIndemnizar al dueño o consignatario de la carga por los daños soportados por la destrucción o pérdida de su mercancía.\n2.\nPagar al Estado los tributos correspondientes.\nEsta responsabilidad se extiende a los tributos que hubieren correspondido a las mercancías que sufran cualquier siniestro, robo o hurto durante su traslado desde el puerto, aeropuerto o frontera de arribo, hasta las bodegas de depósito.\nb.\nEs responsabilidad del dueño, consignatario o el consignante de las mercancías indemnizar por los daños y perjuicios causados en las bodegas, por la naturaleza o peligro de sus mercancías, cuando no hubiere manifestado estas condiciones en los documentos de embarque, o informado expresamente de ellas a los propietarios o empresas autorizadas para bodegas destinadas a depósito temporal y depósitos aduaneros.\n

### 4. Get ready data to reranking
- Requested

[('Query', 'Paragraph1'), ('Query', 'Paragraph2') , ('Query', 'Paragraph3')]

In [19]:
rerank_data = [(query, doc) for doc in retrieval_vector_db_documents]

In [20]:
rerank_data

[('¿Cuál es el objetivo principal del Código Orgánico de la Producción, Comercio e Inversiones? ',
  'a.\nLas responsabilidades de las personas autorizadas para el funcionamiento de las bodegas destinadas a depósito temporal y depósitos aduaneros, son las siguientes:\n1.\nIndemnizar al dueño o consignatario de la carga por los daños soportados por la destrucción o pérdida de su mercancía.\n2.\nPagar al Estado los tributos correspondientes.\nEsta responsabilidad se extiende a los tributos que hubieren correspondido a las mercancías que sufran cualquier siniestro, robo o hurto durante su traslado desde el puerto, aeropuerto o frontera de arribo, hasta las bodegas de depósito.\nb.\nEs responsabilidad del dueño, consignatario o el consignante de las mercancías indemnizar por los daños y perjuicios causados en las bodegas, por la naturaleza o peligro de sus mercancías, cuando no hubiere manifestado estas condiciones en los documentos de embarque, o informado expresamente de ellas a los prop

load model required to get scores

In [25]:
from sentence_transformers import CrossEncoder
model = CrossEncoder('cross-encoder/ms-marco-TinyBERT-L-2', max_length=512)
scores = model.predict(rerank_data)

In [26]:
scores

array([0.0312615 , 0.96338725, 0.3024996 , 0.9156681 , 0.9466539 ,
       0.00117752, 0.956461  , 0.98099077, 0.6581832 , 0.66523373,
       0.9562154 , 0.96338725, 0.97643286, 0.9480292 , 0.8668852 ],
      dtype=float32)

In [27]:
filter_min_score =0
documents_and_scores = [(retrieval_vector_db_documents[i], scores[i]) for i in range(len(retrieval_vector_db_documents)) if scores[i] > filter_min_score]



In [28]:
# Sort the list by scores in descending order (higher scores first)
sorted_documents_and_scores = sorted(documents_and_scores, key=lambda x: x[1], reverse=True)

# Print the documents in order from better to worse based on their scores
doc = []
for document, score in sorted_documents_and_scores:
   doc.append(document)

In [29]:
doc

['LEXISFIND\nel Ministerio de Relaciones Laborales en caso de no existir acuerdo en el referido Consejo.\nLa revisión anual del salario básico se realizará con carácter progresivo hasta alcanzar el salario digno de acuerdo con lo dispuesto en la Constitución de la República y en el presente Código."\n2.\nA continuación del Art. 105, incorpórese el siguiente artículo innumerado:\n"Art. 105.1.- Previo acuerdo del trabajador y del patrono, todo o parte de las utilidades que le corresponde al trabajador, podrán ser canceladas en acciones de la empresa a la que presta sus servicios, siempre que tal empresa se encuentre registrada en una Bolsa de Valores y cumpla con el protocolo para el cumplimiento de ética empresarial definida por el Estado y los requisitos previstos en el Reglamento al Código de la Producción, Comercio e Inversiones."',
 'VIGÉSIMA QUINTA.- Todas las multas por faltas reglamentarias registradas en el Sistema Interactivo de Comercio Exterior (SICE) de la Corporación Aduane

In [30]:
retrieval_vector_db_documents

['a.\nLas responsabilidades de las personas autorizadas para el funcionamiento de las bodegas destinadas a depósito temporal y depósitos aduaneros, son las siguientes:\n1.\nIndemnizar al dueño o consignatario de la carga por los daños soportados por la destrucción o pérdida de su mercancía.\n2.\nPagar al Estado los tributos correspondientes.\nEsta responsabilidad se extiende a los tributos que hubieren correspondido a las mercancías que sufran cualquier siniestro, robo o hurto durante su traslado desde el puerto, aeropuerto o frontera de arribo, hasta las bodegas de depósito.\nb.\nEs responsabilidad del dueño, consignatario o el consignante de las mercancías indemnizar por los daños y perjuicios causados en las bodegas, por la naturaleza o peligro de sus mercancías, cuando no hubiere manifestado estas condiciones en los documentos de embarque, o informado expresamente de ellas a los propietarios o empresas autorizadas para bodegas destinadas a depósito temporal y depósitos aduaneros.\n

### 5_ generate Response

In [31]:
import ollama

In [36]:
output = ollama.generate(
            model="llama3.1",
            prompt=f" Dada la siguiente informacion {doc} responda la siguiente pregunta {query}. \
                No use inormacion previa, tampoco tu previo conocimiento. \
                Si no recibes informacion relacionada con las pregunta en la informacion dada, dame una respuesta corta o unicamente que no tienes informacion sobre lo preguntado. \
                Porfavor se consistente y provee la respuesta mas corta que puedas. No agregues ningun otro mensaje que no este en la informacion dada. Usa Español para responder"
            )

In [37]:
output

{'model': 'llama3.1',
 'created_at': '2024-08-28T02:36:07.934665996Z',
 'response': 'No tengo información sobre el objetivo principal del Código Orgánico de la Producción, Comercio e Inversiones.',
 'done': True,
 'done_reason': 'stop',
 'context': [128009,
  128006,
  882,
  128007,
  271,
  423,
  2649,
  1208,
  56001,
  6179,
  5928,
  2570,
  36345,
  1669,
  37,
  5358,
  1734,
  301,
  9675,
  822,
  409,
  6649,
  12712,
  15277,
  3916,
  665,
  23884,
  409,
  912,
  3073,
  404,
  63960,
  665,
  658,
  8464,
  5362,
  67566,
  7453,
  7255,
  77,
  8921,
  17951,
  3739,
  459,
  940,
  1624,
  94191,
  81451,
  4042,
  513,
  42658,
  1995,
  390,
  1841,
  1995,
  75169,
  463,
  18297,
  6632,
  29626,
  76871,
  57000,
  658,
  94191,
  28677,
  78,
  409,
  63960,
  390,
  781,
  13262,
  32591,
  665,
  1208,
  76704,
  36323,
  409,
  1208,
  3402,
  69174,
  379,
  665,
  658,
  54272,
  356,
  33843,
  44043,
  77,
  17,
  7255,
  77,
  32,
  16513,
  5840,
  1624,

In [38]:
output1 = ollama.generate(
            model="llama3.1",
            prompt=f" Dada la siguiente informacion {doc} responda la siguiente pregunta {query}. "
            )

In [39]:
output1

{'model': 'llama3.1',
 'created_at': '2024-08-28T02:36:40.891649655Z',
 'response': 'Lo siento, pero no pude encontrar una respuesta específica en el texto proporcionado. Sin embargo, puedo intentar ayudarte a determinar el objetivo principal del Código Orgánico de la Producción, Comercio e Inversiones.\n\nEl Código Orgánico de la Producción, Comercio e Inversiones parece ser un conjunto de leyes y regulaciones que rigen las actividades comerciales, productivas e inversionistas en Ecuador. En este sentido, el objetivo principal podría ser establecer una estructura legal clara para facilitar y regular la producción, el comercio y las inversiones dentro del país.\n\nAlgunas posibles respuestas al tema de tu pregunta podrían incluir:\n\n*   Establecer un marco legal claro para las actividades económicas en Ecuador.\n*   Fomentar el crecimiento económico y la competitividad a través de la regulación y promoción de la producción, el comercio e inversiones.\n*   Proteger los derechos de los 